# bt

> bluetooth scripts

## Dependencies:

- pybluez
- pyobjc
- python-lightblue

In [ ]:
# | default_exp bt

## Usage:

In [ ]:
# | export
import bluetooth as bt

In [ ]:
dd = bt.DeviceDiscoverer()

NotImplementedError: 